In [5]:

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark_config = {
    "spark.driver.memory": "4g",
    "spark.executor.memory": "6g",
}

spark_builder = SparkSession.builder.appName("pokemon-etl")

for key, val in spark_config.items():
    spark_builder.config(key, val)
spark = spark_builder.getOrCreate()

23/03/19 14:00:22 WARN Utils: Your hostname, tamsin resolves to a loopback address: 127.0.1.1; using 10.0.0.8 instead (on interface enp34s0)
23/03/19 14:00:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 14:00:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/19 14:00:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
pokemon_extract_path = '../data/transformed/pokemon'
generation_extract_path = '../data/transformed/generation'
evolution_extract_path = '../data/transformed/evolution'

generation_coverage_path = '../data/transformed/generation_coverage'
generation_stats_path = '../data/transformed/generation_stats'
hidden_ability_path = '../data/transformed/hidden_ability'
cosine_similarity_path = '../data/transformed/cosine_similarity'

In [4]:
df_pokemon = spark.read.parquet(pokemon_extract_path)
df_generation = spark.read.parquet(generation_extract_path)
df_evolution = spark.read.parquet(evolution_extract_path)

generation_coverage = spark.read.parquet(generation_coverage_path)
generation_stats = spark.read.parquet(generation_stats_path)
hidden_ability = spark.read.parquet(hidden_ability_path)
cosine_similarity = spark.read.parquet(cosine_similarity_path)

1. What’s the <b>average for each base stats for every Pokémon type across all generations</b>?
    - Need to know all Pokemon names.
    - Need to know all Pokemon types.
    - Need to know all generations.
    - Need to know all base stat types.

Which Pokemon has the highest overall stats? What about for each stat? What about the lowest?
- All stats:
	- Highest: Arcanine - 92.5
	- Lowest: Caterpie - 32.5

- Stat hp:
	- Highest: Wigglytuff - 140
	- Lowest: Diglett - 10

- Stat attack:
	- Highest: Machamp - 130
	- Lowest: Abra - 20

- Stat defense:
	- Highest: Cloyster - 180
	- Lowest: Abra - 15

- Stat special attack:
	- Highest: Alakazam - 135
	- Lowest: Caterpie - 20

- Stat special defense:
	- Highest: Tentacruel - 120
	- Lowest: Caterpie - 20

- Stat speed:
	- Highest: Dugtrio - 120
	- Lowest: Slowpoke - 15


In [5]:
generation_stats.show()

+-------------+-----------------+------------------------+------------------+-----------------+-----------------+-------------------------+-----------------+
|generation_id|  avg(stat_speed)|avg(stat_special_attack)|avg(avg_all_stats)|     avg(stat_hp)|avg(stat_defense)|avg(stat_special_defense)| avg(stat_attack)|
+-------------+-----------------+------------------------+------------------+-----------------+-----------------+-------------------------+-----------------+
|            1|68.63513513513513|       66.87837837837837| 67.62274774774777|64.29729729729729|68.26351351351352|        65.29729729729729|72.36486486486487|
|            2|61.12765957446808|      63.244680851063826| 66.90248226950354|70.51063829787235|68.65957446808511|        70.09574468085107|67.77659574468085|
|            3|61.50769230769231|       67.63076923076923| 67.42564102564101|66.53076923076924|69.16153846153846|        66.94615384615385|72.77692307692308|
|            4|68.60606060606061|       72.050505050

Which generation has the speediest pokemon?

In [6]:
generation_stats.createOrReplaceTempView("avg_stats")
spark.sql(
    """
    select generation_id, `avg(stat_speed)`
    from avg_stats
    where `avg(stat_speed)` in (
        select  max(`avg(stat_speed)`)
        from avg_stats
        );
    """
).show()

+-------------+-----------------+
|generation_id|  avg(stat_speed)|
+-------------+-----------------+
|            9|75.03883495145631|
+-------------+-----------------+



What of the slowest?

In [7]:
spark.sql(
    """
    select generation_id, `avg(stat_speed)`
    from avg_stats
    where `avg(stat_speed)` in (
        select  min(`avg(stat_speed)`)
        from avg_stats
        );
    """
).show()

+-------------+-----------------+
|generation_id|  avg(stat_speed)|
+-------------+-----------------+
|            2|61.12765957446808|
+-------------+-----------------+



It looks like the earlier generation Pokemons are rather weak compared to newer ones. This is a classic case of 'power creep', as in most games.

In [9]:
temp_cols = generation_stats.columns
temp_cols.remove("generation_id")

for stat in temp_cols:
    max = generation_stats.select(F.max(F.col(stat))).collect()[0][0]
    min = generation_stats.select(F.min(F.col(stat))).collect()[0][0]
    name_max = (
        generation_stats.where(F.col(stat) == F.lit(max))
        .select(F.col("generation_id"))
        .collect()[0][0]
        #.capitalize()
    )
    name_min = (
        generation_stats.where(F.col(stat) == F.lit(min))
        .select(F.col("generation_id"))
        .collect()[0][0]
        #.capitalize()
    )
    stat_pretty = (
        stat.replace("avg", "")
        .replace("_", " ")
        .replace("(", "")
        .replace(")", "")
        .capitalize()
    )
    print(
        f"{stat_pretty}:\n\tHighest: {name_max} - {int(max)}\n\tLowest: {name_min} - {int(min)}"
    )
    print()


Stat speed:
	Highest: 9 - 75
	Lowest: 2 - 61

Stat special attack:
	Highest: 7 - 75
	Lowest: 2 - 63

 all stats:
	Highest: 7 - 74
	Lowest: 2 - 66

Stat hp:
	Highest: 9 - 75
	Lowest: 1 - 64

Stat defense:
	Highest: 7 - 79
	Lowest: 1 - 68

Stat special defense:
	Highest: 7 - 74
	Lowest: 1 - 65

Stat attack:
	Highest: 7 - 85
	Lowest: 2 - 67



2. Find the <b>Pokémon with the best type coverage per generation</b>.
    - Need to algorithmically define the coverage = can learn the most amount of moves with different types.
    - Need to know the generations.

Again, we see that G7 emerges as the winner in terms of coverage. A trend?

In [22]:
generation_coverage.show()

+-------------+------------------+
|generation_id|      avg_coverage|
+-------------+------------------+
|            1|0.6058558558558563|
|            2|0.6063829787234043|
|            3|0.7410256410256406|
|            4|0.7020202020202018|
|            5|0.6845637583892626|
|            6|0.7828282828282829|
|            7|1.0081300813008132|
|            8|0.6567460317460314|
|            9|0.7135922330097088|
+-------------+------------------+



1. For all abilities find all Pokémon that have the ability as <b>a hidden ability</b> and all Pokémon that have it as <b>a non-hidden ability</b>.
    - Need to know all abilities.
    - Need to break them down into hidden and not hidden abilities.

In [23]:
hidden_ability.show()

+-------------+--------------------+--------------------+
| ability_name|              hidden|          not_hidden|
+-------------+--------------------+--------------------+
| adaptability|[skrelp, crawdaun...|  [eevee, porygon-z]|
|    aftermath|[trubbish, garbod...|[drifblim, driflo...|
|  anger-point|[crabrawler, sand...|[mankey, primeape...|
| anticipation| [eevee, ferrothorn]|[hatterene, croag...|
|   aroma-veil|[spritzee, aromat...|           [lechonk]|
| battle-armor|   [cubone, marowak]|[skorupi, anorith...|
|beads-of-ruin|            [chi-yu]|            [chi-yu]|
|    big-pecks|[pidgey, pidgeott...|[pidove, bombirdi...|
|        blaze| [simisear, pansear]|[chimchar, emboar...|
|  bulletproof|[applin, dubwool,...|[hakamo-o, ursalu...|
|  chlorophyll|[bulbasaur, cotto...|[vileplume, victr...|
|   clear-body|[klang, naclstack...|[metang, drakloak...|
|   cloud-nine|[drampa, lickilic...|  [psyduck, golduck]|
|  competitive|[kilowattrel, wat...|[igglybuff, jiggl...|
|     contrary

2. Find Pokémon with similar move-set that do not belong to the same evolution chain and are found in different generations.
    - Need to define move set = set of abilities.
    - Need to know evolution chains and generations.

For this task, I thought of transforming the abilities list of strings into a vector of floats, using the TF-IDF method from Statistics/ML. This way, we can calculate a similarity score, the Cosine Similarity (goes from 0 to 1) between two vectors. Then we join the TF-IDF dataframe with itself in an (expensive) Cartesian Join, in order to compute the similarity score between each Pokemon species. Then we use a lower threshold of 0.75 similarity to filter out this massive dataset. The Pokemon pairs are in different generations and evolution chains.

In [25]:
cosine_similarity.show()

+----------+----------+------------------+
|    name_i|    name_j|        sim_cosine|
+----------+----------+------------------+
|aerodactyl| vespiquen|0.7951824850842304|
|     aipom|   pikipek|0.7661804024859917|
|     aipom|  trumbeak|0.7661804024859917|
|   anorith|    kabuto|0.8247689672422843|
|   anorith|  kabutops|0.8247689672422843|
|   anorith| type-null|0.8212873335251791|
|     arbok|masquerain|0.7845044061125963|
|   ariados|  beedrill|0.7899283003409685|
|   armaldo|    kabuto|0.8247689672422843|
|   armaldo|  kabutops|0.8247689672422843|
|   armaldo| type-null|0.8212873335251791|
|     azelf|    baltoy|               1.0|
|     azelf|  chimecho|               1.0|
|     azelf|   claydol|               1.0|
|     azelf| cryogonal|               1.0|
|     azelf| eelektrik|               1.0|
|     azelf|eelektross|               1.0|
|     azelf|    flygon|               1.0|
|     azelf|    gastly|               1.0|
|     azelf|   haunter|               1.0|
+----------